### This notebook use the clean the data from yahoo finance

In [17]:
import pandas as pd
from pathlib import Path
import numpy as np
import json



In [18]:
from FinancialStatementSearch import get_financial_data

ModuleNotFoundError: No module named 'FinancialStatementSearch'

In [ ]:
'''Get the name and industry of the ASX data first'''

# Path to your JSON file
json_path = Path("asx_companies_info.json")  

# Read the Json file
with open(json_path, "r", encoding="utf-8") as f:
    companies = json.load(f)


# Convert to DataFrame
df = pd.DataFrame(companies)

df = df.drop(columns=['name']) # 11 sectors, 75 industries
 
df = df[df['sector'].notna()]

For Balance Sheet, we only care about the latest date, which is the first column

In [ ]:

# --- Paths ---
company_data_path = Path("company_data")
company_data_path.mkdir(parents=True, exist_ok=True)

# Print existing directories (companies)
for ticker in company_data_path.iterdir():
    if ticker.is_dir():
        print(ticker.name)

# --- Loop through tickers ---
for base in (
    df["ticker"]
    .dropna()
    .astype(str)
    .str.strip()
    .replace("", np.nan)
    .dropna()
    .unique()
):
    ticker = f"{base.upper()}.AX"
    ticker_dir = company_data_path / ticker

    # Skip if folder already exists
    if ticker_dir.exists():
        print(f"Folder already exists for {ticker} — skipping")
        continue

    print(f"\n📊 Processing: {ticker}")

    # Create folder for the company
    ticker_dir.mkdir(parents=True, exist_ok=True)

    # Expected JSON file path
    json_path = ticker_dir / "financial_data.json"

    # Fetch and save data
    try:
        data = get_financial_data(ticker)
        if data is not None:
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)
            print(f"💾 Saved data to {json_path}")
        else:
            print(f"⚠ No data returned for {ticker}")
    except Exception as e:
        print(f" Error fetching {ticker}: {e}")

NUF.AX
LLC.AX
NAB.AX
DXS.AX
SPK.AX
IEL.AX
HLS.AX
IPH.AX
PTM.AX
PMV.AX
PPT.AX
NEC.AX
BRG.AX
ASX.AX
ANZ.AX
IFL.AX
DOW.AX
SUN.AX
SHL.AX
PNV.AX
SEK.AX
LYC.AX
APX.AX
CGF.AX
ORG.AX
APA.AX
AMC.AX
PLS.AX
FLT.AX
STO.AX
APE.AX
SIQ.AX
SFR.AX
SXL.AX
SGR.AX
SOL.AX
WBC.AX
ELD.AX
HUB.AX
S32.AX
MTS.AX
NWH.AX
MGR.AX
CSL.AX
ALL.AX
IAG.AX
RHC.AX
GOZ.AX
BWP.AX
AMP.AX
NWL.AX
BAP.AX
GWA.AX
FMG.AX
QUB.AX
SKC.AX
REA.AX
GMG.AX
GNC.AX
CCL.AX
ALQ.AX
CMW.AX
CPU.AX
BPT.AX
CLW.AX
NST.AX
INA.AX
PME.AX
BSL.AX
ORI.AX
BKW.AX
CCP.AX
JHX.AX
RWC.AX
NAN.AX
ANN.AX
BOQ.AX
MPL.AX
COH.AX
AZJ.AX
RMD.AX
AGL.AX
COL.AX
OML.AX
BHP.AX
CQR.AX
IRE.AX
DHG.AX
SDF.AX
NWS.AX
AVH.AX
CNU.AX
TCL.AX
NSR.AX
MMS.AX
BEN.AX
TAH.AX
QBE.AX
RSG.AX
ING.AX
CAR.AX
A2M.AX
NXT.AX
CUV.AX
TLS.AX
MIN.AX
BVS.AX
SCG.AX
ALX.AX
GOR.AX
RIO.AX
SUL.AX
SGM.AX
JBH.AX
GPT.AX
QAN.AX
PRN.AX
CWY.AX
MYX.AX
NHF.AX
MND.AX
BGA.AX
ASB.AX
HVN.AX
JIN.AX
SCP.AX
SSM.AX
RRL.AX
EML.AX
CHC.AX
ARB.AX
CBA.AX
ORA.AX
PNI.AX
SGP.AX
FPH.AX
GEM.AX
CKF.AX
CTD.AX
ORE.AX
NHC.AX
SBM.AX
FBU.AX

KeyboardInterrupt: 

In [ ]:
from pathlib import Path
import pandas as pd

base_dir = Path("company_data")
all_balance_sheets = []

balance_sheet_standard_columns = [
    "Total assets",
    "Total liabilities net minority interest",
    "Total equity gross minority interest",
    "Total capitalisation",
    "Common stock equity",
    "Net tangible assets",
    "Invested capital",
    "Tangible book value",
    "Total debt",
    "Share issued",
    "Ordinary shares number",
]

# Loop through every company in the company folder data
for company_folder in base_dir.iterdir():
    if not company_folder.is_dir() or company_folder.name.startswith("."):
        continue  # skip files like .DS_Store and hidden items

    # folder name only
    ticker_with_suffix = company_folder.name

    # remove ".AX" only if it's at the end
    ticker = ticker_with_suffix[:-3] if ticker_with_suffix.endswith(".AX") else ticker_with_suffix

    # load
    bs_path = company_folder / "BalanceSheet.csv"
    if not bs_path.exists():
        continue
    balance_sheet = pd.read_csv(bs_path)

    
    # Only care for the latest data
    latest_col = balance_sheet.columns[1]

    company_balance_sheet = (
    balance_sheet[["Breakdown", latest_col]]
        .assign(
            Value=lambda x: (
                x[latest_col]
                .replace("--", np.nan)               # turn "--" into NaN
                .str.replace(",", "", regex=False)   # strip commas
                .str.replace("\u2212", "-", regex=False)  # fix unicode minus
                .astype(float)
            )
        )
        .pivot_table(index=None, columns="Breakdown", values="Value")
        .rename_axis(None, axis=1)
        .reset_index(drop=True)
    )

    company_balance_sheet = company_balance_sheet[balance_sheet_standard_columns]
    # Set the ticker for each company, make it as an index
    company_balance_sheet.insert(0, "Ticker", ticker)
    company_balance_sheet = company_balance_sheet.set_index("Ticker")
    all_balance_sheets.append(company_balance_sheet)

combined_balance_sheets = pd.concat(all_balance_sheets)



In [ ]:
combined_balance_sheets = combined_balance_sheets.merge(
    df,
    left_index=True,     
    right_on="ticker",   
    how="left"
)
# Make the tikcer as the index 
combined_balance_sheets = combined_balance_sheets.set_index("ticker")

In [ ]:
balance_sheet_sector_means = combined_balance_sheets.groupby("sector").mean(numeric_only=True)

In [ ]:
balance_sheet_sector_means

,Total assets,Total liabilities net minority interest,Total equity gross minority interest,Total capitalisation,Common stock equity,Net tangible assets,Invested capital,Tangible book value,Total debt,Share issued,Ordinary shares number
sector,,,,,,,,,,,
Basic Materials,1.520879e+07,6.902871e+06,8.305920e+06,1.016226e+07,7.896392e+06,7.313682e+06,1.027441e+07,7.313682e+06,2.807929e+06,1.367215e+06,1.366630e+06
Communication Services,5.005517e+06,2.570748e+06,2.434769e+06,3.337295e+06,2.331329e+06,3.511306e+05,3.423677e+06,3.511306e+05,1.507967e+06,6.917053e+05,6.799498e+05
Consumer Cyclical,5.361022e+06,3.146314e+06,2.219317e+06,3.429508e+06,2.212759e+06,9.279329e+04,3.589987e+06,9.279329e+04,1.845784e+06,5.435111e+05,5.434817e+05
Consumer Defensive,4.203471e+06,3.011542e+06,1.187169e+06,1.558428e+06,1.187544e+06,5.279997e+05,1.687818e+06,5.279997e+05,1.720456e+06,5.298569e+05,5.298569e+05
Energy,1.281782e+07,5.709613e+06,7.108208e+06,8.907775e+06,7.108208e+06,6.660401e+06,9.336630e+06,6.660401e+06,2.544116e+06,2.123969e+06,2.123969e+06
Financial Services,2.346745e+08,2.186676e+08,1.600689e+07,5.511232e+07,1.588795e+07,1.324896e+07,5.757863e+07,1.324896e+07,4.203282e+07,1.229914e+06,1.227835e+06
Healthcare,7.058303e+06,3.542753e+06,3.515550e+06,4.690901e+06,3.306556e+06,5.054798e+05,4.761621e+06,5.054798e+05,2.275667e+06,2.973337e+05,2.941960e+05
Industrials,6.913276e+06,4.737523e+06,2.175753e+06,4.207607e+06,1.980636e+06,8.210039e+04,4.441501e+06,8.210039e+04,2.797535e+06,9.702194e+05,9.702194e+05
Real Estate,1.181790e+07,4.728202e+06,7.089703e+06,1.000557e+07,6.866315e+06,6.705457e+06,1.049564e+07,6.705457e+06,3.670907e+06,1.660069e+06,1.660069e+06


### Company cash flow


In [ ]:
base_dir = Path("company_data")
all_cash_flows = []

for company_folder in base_dir.iterdir():
    if not company_folder.is_dir() or company_folder.name.startswith("."):
        continue  # skip .DS_Store etc.

    ticker = company_folder.name[:-3] if company_folder.name.endswith(".AX") else company_folder.name
    cf_path = company_folder / "CashFlow.csv"
    if not cf_path.exists():
        continue

    cash_flow = pd.read_csv(cf_path)
    latest_col = cash_flow.columns[1]

    company_cash_flow = (
        cash_flow[["Breakdown", latest_col]]
          .assign(
              Value=lambda x: (
                  x[latest_col]
                  .replace("--", np.nan)
                  .str.replace(",", "", regex=False)
                  .str.replace("\u2212", "-", regex=False)
                  .astype(float)
              )
          )
          .pivot_table(index=None, columns="Breakdown", values="Value")
          .rename_axis(None, axis=1)
          .reset_index(drop=True)
    )

    company_cash_flow.insert(0, "Ticker", ticker)
    company_cash_flow = company_cash_flow.set_index("Ticker")
    all_cash_flows.append(company_cash_flow)

# concat with alignment — mismatched columns will be filled with NaN
combined_cash_flows = pd.concat(all_cash_flows, sort=True)

In [ ]:
combined_cash_flows = combined_cash_flows.merge(
    df,
    left_index=True,     
    right_on="ticker",   
    how="left"
)
# Make the tikcer as the index 
combined_cash_flows = combined_cash_flows.set_index("ticker")

In [ ]:
combined_cash_flows.to_csv("cash_flow.csv")

In [ ]:
cash_flow_means_sector = combined_cash_flows.groupby("sector").mean(numeric_only=True)

In [ ]:
cash_flow_means_sector

,All taxes paid,Beginning cash position,Capital expenditure,Cash flow from discontinued operation,Cash flows from used in operating activities direct,Cash payments for deposits by banks & customers,Cash payments for loans,Cash receipts from deposits by banks & customers,Cash receipts from loans,Cash receipts from securities-related activities,...,Interest and commission paid,Interest paid supplemental data,Investing cash flow,Issuance of capital stock,Issuance of debt,Operating cash flow,Other cash payments from operating activities,Other cash receipts from operating activities,Repayment of debt,Repurchase of capital stock
sector,,,,,,,,,,,,,,,,,,,,,
Basic Materials,NaN,NaN,-1.431921e+06,NaN,8.848009e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,1.097771e+05,-1.864518e+06,1.747534e+05,2.040294e+06,7.281354e+06,NaN,NaN,-1.223825e+06,-62398.500000
Communication Services,NaN,NaN,-2.024558e+05,370000.0,4.299681e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,8.292157e+04,-1.448919e+05,5.789267e+04,3.091789e+05,9.780000e+05,NaN,NaN,-3.328520e+05,-37270.400000
Consumer Cyclical,NaN,NaN,-1.393038e+05,NaN,3.894869e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,7.350327e+04,-2.663502e+05,1.062769e+05,3.620476e+05,1.390000e+06,NaN,NaN,-2.594879e+05,-80621.800000
Consumer Defensive,NaN,NaN,-2.161197e+05,NaN,4.709747e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,9.197420e+04,-2.478510e+05,1.493005e+05,5.690706e+05,NaN,NaN,NaN,-5.158000e+05,-10552.001286
Energy,NaN,NaN,-1.123931e+06,NaN,1.555101e+06,NaN,NaN,NaN,NaN,NaN,...,NaN,5.184667e+04,-1.256483e+06,NaN,3.950000e+05,NaN,NaN,NaN,-7.895367e+04,-10463.000000
Financial Services,-4197000.0,47321000.0,-2.723318e+05,NaN,-8.273695e+04,-1438000.0,-75875000.0,62015000.0,74592000.0,41123000.0,...,-40405000.0,1.097505e+06,-2.558398e+06,1.403479e+05,1.592433e+07,7.500108e+06,-60709000.0,4069000.0,-1.130389e+07,-606537.555556
Healthcare,NaN,NaN,-1.973760e+05,NaN,6.555681e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,1.194198e+05,-1.929361e+05,1.593047e+05,1.550629e+06,8.578775e+05,NaN,NaN,-1.490471e+06,-58469.818333
Industrials,NaN,NaN,-4.383796e+05,NaN,6.807941e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,1.122196e+05,-3.524393e+05,3.898720e+05,7.348574e+05,NaN,NaN,NaN,-6.231609e+05,-141568.000000
Real Estate,NaN,NaN,-2.679917e+04,NaN,3.231082e+05,NaN,NaN,NaN,NaN,NaN,...,NaN,1.970150e+05,-2.684419e+05,1.187095e+06,2.625875e+06,NaN,NaN,NaN,-2.381556e+06,-34217.777778


### Financial Data

In [ ]:
base_dir = Path("company_data")
all_financial_data = []

for company_folder in base_dir.iterdir():
    if not company_folder.is_dir() or company_folder.name.startswith("."):
        continue  

    ticker = company_folder.name[:-3] if company_folder.name.endswith(".AX") else company_folder.name

    fd_path = company_folder / "financial_data.csv"
    if not fd_path.exists():
        fd_path = company_folder / "IncomeStatement.csv"
    if not fd_path.exists():
        continue

    financial_data = pd.read_csv(fd_path)
    latest_col = financial_data.columns[1]

    company_financial_data = (
        financial_data[["Breakdown", latest_col]]
          .assign(
              Value=lambda x: (
                  x[latest_col]
                  .replace("--", np.nan)
                  .str.replace(",", "", regex=False)
                  .str.replace("\u2212", "-", regex=False)
                  .astype(float)
              )
          )
          .pivot_table(index=None, columns="Breakdown", values="Value")
          .rename_axis(None, axis=1)
          .reset_index(drop=True)
    )

    company_financial_data.insert(0, "Ticker", ticker)
    company_financial_data = company_financial_data.set_index("Ticker")
    all_financial_data.append(company_financial_data)

# concat with alignment — mismatched columns will be NaN
combined_financial_data = pd.concat(all_financial_data, sort=True)




In [ ]:
combined_financial_data

,Average dilution earnings,Basic EPS,Basic average shares,Cost of revenue,Credit losses provision,Diluted EPS,Diluted NI available to com stockholders,Diluted average shares,EBIT,EBITDA,...,Special income charges,Tax effect of unusual items,Tax provision,Tax rate for calcs,Total expenses,Total money market investments,Total operating income as reported,Total revenue,Total unusual items,Total unusual items excluding goodwill
Ticker,,,,,,,,,,,,,,,,,,,,,
NUF,NaN,-0.07,382588.436,2515210.0,NaN,-0.07,NaN,385830.981,53938.0,271277.0,...,NaN,0.00,-16461.0,0.0004,NaN,NaN,NaN,3398748.0,NaN,NaN
LLC,NaN,0.33,681000.000,7199000.0,NaN,0.33,NaN,681000.000,646000.0,739000.0,...,NaN,4200.55,139000.0,0.0000,NaN,NaN,NaN,7603000.0,11000.00,11000.00
NAB,403000.0,2.24,3074500.000,NaN,NaN,2.21,7276000.0,3285000.000,NaN,NaN,...,-85000.0,-24467.86,2821000.0,0.0000,NaN,NaN,NaN,20702000.0,-85000.00,-85000.00
DXS,NaN,0.15,1073620.120,219900.0,NaN,0.15,NaN,1073620.120,337600.0,355700.0,...,NaN,-38160.00,-1400.0,0.0000,NaN,NaN,NaN,921200.0,-127200.00,-127200.00
SPK,NaN,0.14,1847000.000,1809000.0,NaN,0.14,NaN,1847000.000,445000.0,1035000.0,...,NaN,2192.00,95000.0,0.0003,NaN,NaN,NaN,3623000.0,8000.00,8000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMP,NaN,-0.04,92593.000,995465.0,NaN,-0.04,NaN,92593.000,31204.0,179739.0,...,NaN,-22820.70,11472.0,0.0000,NaN,NaN,NaN,2296878.0,-76069.00,-76069.00
IGO,NaN,-1.26,757267.810,NaN,NaN,-1.26,NaN,757267.810,-1037200.0,-811100.0,...,NaN,79.19,-92100.0,0.0000,NaN,NaN,NaN,512500.0,900.00,900.00
BXB,NaN,0.65,1383100.000,3361600.0,NaN,0.65,NaN,1389147.290,1324100.0,2147200.0,...,NaN,0.00,369900.0,0.0000,NaN,NaN,NaN,6669700.0,-40798.48,-40798.48


In [ ]:
combined_financial_data = combined_financial_data.merge(
    df,
    left_index=True,     
    right_on="ticker",   
    how="left"
)
# Make the tikcer as the index 
combined_financial_data = combined_financial_data.set_index("ticker")

In [ ]:
combined_financial_data

,Average dilution earnings,Basic EPS,Basic average shares,Cost of revenue,Credit losses provision,Diluted EPS,Diluted NI available to com stockholders,Diluted average shares,EBIT,EBITDA,...,Tax provision,Tax rate for calcs,Total expenses,Total money market investments,Total operating income as reported,Total revenue,Total unusual items,Total unusual items excluding goodwill,industry,sector
ticker,,,,,,,,,,,,,,,,,,,,,
NUF,NaN,-0.07,382588.436,2515210.0,NaN,-0.07,NaN,385830.981,53938.0,271277.0,...,-16461.0,0.0004,NaN,NaN,NaN,3398748.0,NaN,NaN,Agricultural Inputs,Basic Materials
LLC,NaN,0.33,681000.000,7199000.0,NaN,0.33,NaN,681000.000,646000.0,739000.0,...,139000.0,0.0000,NaN,NaN,NaN,7603000.0,11000.00,11000.00,Real Estate - Diversified,Real Estate
NAB,403000.0,2.24,3074500.000,NaN,NaN,2.21,7276000.0,3285000.000,NaN,NaN,...,2821000.0,0.0000,NaN,NaN,NaN,20702000.0,-85000.00,-85000.00,Banks - Diversified,Financial Services
DXS,NaN,0.15,1073620.120,219900.0,NaN,0.15,NaN,1073620.120,337600.0,355700.0,...,-1400.0,0.0000,NaN,NaN,NaN,921200.0,-127200.00,-127200.00,REIT - Office,Real Estate
SPK,NaN,0.14,1847000.000,1809000.0,NaN,0.14,NaN,1847000.000,445000.0,1035000.0,...,95000.0,0.0003,NaN,NaN,NaN,3623000.0,8000.00,8000.00,Telecom Services,Communication Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMP,NaN,-0.04,92593.000,995465.0,NaN,-0.04,NaN,92593.000,31204.0,179739.0,...,11472.0,0.0000,NaN,NaN,NaN,2296878.0,-76069.00,-76069.00,Restaurants,Consumer Cyclical
IGO,NaN,-1.26,757267.810,NaN,NaN,-1.26,NaN,757267.810,-1037200.0,-811100.0,...,-92100.0,0.0000,NaN,NaN,NaN,512500.0,900.00,900.00,Other Industrial Metals & Mining,Basic Materials
BXB,NaN,0.65,1383100.000,3361600.0,NaN,0.65,NaN,1389147.290,1324100.0,2147200.0,...,369900.0,0.0000,NaN,NaN,NaN,6669700.0,-40798.48,-40798.48,Specialty Business Services,Industrials


In [ ]:
combined_financial_data.to_csv("financial_data.csv")

In [ ]:
financial_data_means_sector = combined_financial_data.groupby("sector").mean(numeric_only=True)

In [ ]:
no_null_cols

Index(['Basic EPS', 'Basic average shares', 'Diluted EPS',
       'Diluted average shares', 'Net income common stockholders',
       'Net income from continuing & discontinued operation',
       'Net income from continuing operation net minority interest',
       'Normalised income', 'Pre-tax income', 'Tax effect of unusual items',
       'Tax provision', 'Tax rate for calcs', 'Total revenue', 'industry',
       'sector'],
      dtype='object')

In [ ]:
balance_sheet_sector_means

,Total assets,Total liabilities net minority interest,Total equity gross minority interest,Total capitalisation,Common stock equity,Net tangible assets,Invested capital,Tangible book value,Total debt,Share issued,Ordinary shares number
sector,,,,,,,,,,,
Basic Materials,1.520879e+07,6.902871e+06,8.305920e+06,1.016226e+07,7.896392e+06,7.313682e+06,1.027441e+07,7.313682e+06,2.807929e+06,1.367215e+06,1.366630e+06
Communication Services,5.005517e+06,2.570748e+06,2.434769e+06,3.337295e+06,2.331329e+06,3.511306e+05,3.423677e+06,3.511306e+05,1.507967e+06,6.917053e+05,6.799498e+05
Consumer Cyclical,5.361022e+06,3.146314e+06,2.219317e+06,3.429508e+06,2.212759e+06,9.279329e+04,3.589987e+06,9.279329e+04,1.845784e+06,5.435111e+05,5.434817e+05
Consumer Defensive,4.203471e+06,3.011542e+06,1.187169e+06,1.558428e+06,1.187544e+06,5.279997e+05,1.687818e+06,5.279997e+05,1.720456e+06,5.298569e+05,5.298569e+05
Energy,1.281782e+07,5.709613e+06,7.108208e+06,8.907775e+06,7.108208e+06,6.660401e+06,9.336630e+06,6.660401e+06,2.544116e+06,2.123969e+06,2.123969e+06
Financial Services,2.346745e+08,2.186676e+08,1.600689e+07,5.511232e+07,1.588795e+07,1.324896e+07,5.757863e+07,1.324896e+07,4.203282e+07,1.229914e+06,1.227835e+06
Healthcare,7.058303e+06,3.542753e+06,3.515550e+06,4.690901e+06,3.306556e+06,5.054798e+05,4.761621e+06,5.054798e+05,2.275667e+06,2.973337e+05,2.941960e+05
Industrials,6.913276e+06,4.737523e+06,2.175753e+06,4.207607e+06,1.980636e+06,8.210039e+04,4.441501e+06,8.210039e+04,2.797535e+06,9.702194e+05,9.702194e+05
Real Estate,1.181790e+07,4.728202e+06,7.089703e+06,1.000557e+07,6.866315e+06,6.705457e+06,1.049564e+07,6.705457e+06,3.670907e+06,1.660069e+06,1.660069e+06


In [ ]:
merged_sector_means = pd.concat(
    [financial_data_means_sector, cash_flow_means_sector, balance_sheet_sector_means],
    axis=1,          # merge side-by-side
    join="inner"     # keep only matching indices (e.g., common sectors)
)

In [ ]:
merged_sector_means

,Average dilution earnings,Basic EPS,Basic average shares,Cost of revenue,Credit losses provision,Diluted EPS,Diluted NI available to com stockholders,Diluted average shares,EBIT,EBITDA,...,Total liabilities net minority interest,Total equity gross minority interest,Total capitalisation,Common stock equity,Net tangible assets,Invested capital,Tangible book value,Total debt,Share issued,Ordinary shares number
sector,,,,,,,,,,,,,,,,,,,,,
Basic Materials,NaN,0.290417,1.359576e+06,3.778645e+06,NaN,0.282917,3.980452e+06,1.365018e+06,1.940869e+06,2.851850e+06,...,6.902871e+06,8.305920e+06,1.016226e+07,7.896392e+06,7.313682e+06,1.027441e+07,7.313682e+06,2.807929e+06,1.367215e+06,1.366630e+06
Communication Services,NaN,0.992222,6.749603e+05,6.447675e+05,NaN,0.990000,1.180000e+06,6.876559e+05,4.127411e+05,6.743937e+05,...,2.570748e+06,2.434769e+06,3.337295e+06,2.331329e+06,3.511306e+05,3.423677e+06,3.511306e+05,1.507967e+06,6.917053e+05,6.799498e+05
Consumer Cyclical,NaN,0.902778,5.473113e+05,2.723146e+06,NaN,0.881667,5.100000e+05,5.489417e+05,3.676349e+05,5.688356e+05,...,3.146314e+06,2.219317e+06,3.429508e+06,2.212759e+06,9.279329e+04,3.589987e+06,9.279329e+04,1.845784e+06,5.435111e+05,5.434817e+05
Consumer Defensive,NaN,0.250000,5.400992e+05,7.814013e+06,NaN,0.250000,NaN,5.408388e+05,3.087263e+05,5.333001e+05,...,3.011542e+06,1.187169e+06,1.558428e+06,1.187544e+06,5.279997e+05,1.687818e+06,5.279997e+05,1.720456e+06,5.298569e+05,5.298569e+05
Energy,NaN,0.273333,2.121963e+06,1.136219e+06,NaN,0.266667,NaN,2.135811e+06,7.751777e+05,1.626037e+06,...,5.709613e+06,7.108208e+06,8.907775e+06,7.108208e+06,6.660401e+06,9.336630e+06,6.660401e+06,2.544116e+06,2.123969e+06,2.123969e+06
Financial Services,254800.0,1.451667,1.058093e+06,9.573717e+05,-428000.0,1.437083,4.694273e+06,1.105742e+06,2.239125e+06,2.698729e+06,...,2.186676e+08,1.600689e+07,5.511232e+07,1.588795e+07,1.324896e+07,5.757863e+07,1.324896e+07,4.203282e+07,1.229914e+06,1.227835e+06
Healthcare,NaN,1.111429,3.958685e+05,1.295118e+06,NaN,1.119286,6.745750e+05,3.982330e+05,5.663616e+05,8.387986e+05,...,3.542753e+06,3.515550e+06,4.690901e+06,3.306556e+06,5.054798e+05,4.761621e+06,5.054798e+05,2.275667e+06,2.973337e+05,2.941960e+05
Industrials,NaN,0.362778,9.702981e+05,1.831713e+06,NaN,0.372222,NaN,9.735184e+05,4.710380e+05,8.404041e+05,...,4.737523e+06,2.175753e+06,4.207607e+06,1.980636e+06,8.210039e+04,4.441501e+06,8.210039e+04,2.797535e+06,9.702194e+05,9.702194e+05
Real Estate,NaN,0.248667,1.654618e+06,9.747248e+05,NaN,0.248000,NaN,1.656843e+06,5.894961e+05,6.059211e+05,...,4.728202e+06,7.089703e+06,1.000557e+07,6.866315e+06,6.705457e+06,1.049564e+07,6.705457e+06,3.670907e+06,1.660069e+06,1.660069e+06


In [ ]:
merged_sector_means.to_csv("sector_means.csv")

In [ ]:
import pandas as pd
sector_means = pd.read_csv("combined_data/sector_means.csv", index_col=0)

In [ ]:
# Remove columns that has NaN
# Remove columns that have at least 1 NaN value
sector_means_clean = sector_means.dropna(axis=1, how='any')

# Display the cleaned DataFrame
sector_means_clean

,Basic EPS,Basic average shares,Cost of revenue,Diluted EPS,Diluted average shares,EBIT,EBITDA,Gross profit,Interest expense,Interest income,...,Total liabilities net minority interest,Total equity gross minority interest,Total capitalisation,Common stock equity,Net tangible assets,Invested capital,Tangible book value,Total debt,Share issued,Ordinary shares number
sector,,,,,,,,,,,,,,,,,,,,,
Basic Materials,0.290417,1.359576e+06,3.778645e+06,0.282917,1.365018e+06,1.940869e+06,2.851850e+06,4.663947e+06,2.237804e+05,5.911825e+04,...,6.902871e+06,8.305920e+06,1.016226e+07,7.896392e+06,7.313682e+06,1.027441e+07,7.313682e+06,2.807929e+06,1.367215e+06,1.366630e+06
Communication Services,0.992222,6.749603e+05,6.447675e+05,0.990000,6.876559e+05,4.127411e+05,6.743937e+05,1.086926e+06,7.045438e+04,9.829125e+03,...,2.570748e+06,2.434769e+06,3.337295e+06,2.331329e+06,3.511306e+05,3.423677e+06,3.511306e+05,1.507967e+06,6.917053e+05,6.799498e+05
Consumer Cyclical,0.902778,5.473113e+05,2.723146e+06,0.881667,5.489417e+05,3.676349e+05,5.688356e+05,1.498701e+06,6.432100e+04,1.332775e+04,...,3.146314e+06,2.219317e+06,3.429508e+06,2.212759e+06,9.279329e+04,3.589987e+06,9.279329e+04,1.845784e+06,5.435111e+05,5.434817e+05
Consumer Defensive,0.250000,5.400992e+05,7.814013e+06,0.250000,5.408388e+05,3.087263e+05,5.333001e+05,1.949641e+06,4.431222e+04,1.147106e+04,...,3.011542e+06,1.187169e+06,1.558428e+06,1.187544e+06,5.279997e+05,1.687818e+06,5.279997e+05,1.720456e+06,5.298569e+05,5.298569e+05
Energy,0.273333,2.121963e+06,1.136219e+06,0.266667,2.135811e+06,7.751777e+05,1.626037e+06,1.918016e+06,1.163757e+05,4.182133e+04,...,5.709613e+06,7.108208e+06,8.907775e+06,7.108208e+06,6.660401e+06,9.336630e+06,6.660401e+06,2.544116e+06,2.123969e+06,2.123969e+06
Financial Services,1.451667,1.058093e+06,9.573717e+05,1.437083,1.105742e+06,2.239125e+06,2.698729e+06,1.385544e+06,1.223702e+06,2.458779e+06,...,2.186676e+08,1.600689e+07,5.511232e+07,1.588795e+07,1.324896e+07,5.757863e+07,1.324896e+07,4.203282e+07,1.229914e+06,1.227835e+06
Healthcare,1.111429,3.958685e+05,1.295118e+06,1.119286,3.982330e+05,5.663616e+05,8.387986e+05,2.701739e+06,8.227474e+04,1.161614e+04,...,3.542753e+06,3.515550e+06,4.690901e+06,3.306556e+06,5.054798e+05,4.761621e+06,5.054798e+05,2.275667e+06,2.973337e+05,2.941960e+05
Industrials,0.362778,9.702981e+05,1.831713e+06,0.372222,9.735184e+05,4.710380e+05,8.404041e+05,2.676978e+06,1.256189e+05,2.618253e+04,...,4.737523e+06,2.175753e+06,4.207607e+06,1.980636e+06,8.210039e+04,4.441501e+06,8.210039e+04,2.797535e+06,9.702194e+05,9.702194e+05
Real Estate,0.248667,1.654618e+06,9.747248e+05,0.248000,1.656843e+06,5.894961e+05,6.059211e+05,5.856637e+05,1.688883e+05,3.618493e+04,...,4.728202e+06,7.089703e+06,1.000557e+07,6.866315e+06,6.705457e+06,1.049564e+07,6.705457e+06,3.670907e+06,1.660069e+06,1.660069e+06


In [ ]:

sector_means_clean.to_csv("combined_data/sector_means_clean.csv")

In [21]:
cash_flow_data = pd.read_csv("combined_data/cash_flow.csv", index_col=0)
cash_flow_data.shape

(142, 30)

In [22]:
cash_flow_data.columns

Index(['All taxes paid', 'Beginning cash position', 'Capital expenditure',
       'Cash flow from discontinued operation',
       'Cash flows from used in operating activities direct',
       'Cash payments for deposits by banks & customers',
       'Cash payments for loans',
       'Cash receipts from deposits by banks & customers',
       'Cash receipts from loans',
       'Cash receipts from securities-related activities', 'Changes in cash',
       'Classes of cash payments',
       'Classes of cash receipts from operating activities',
       'Effect of exchange rate changes', 'End cash position',
       'Financing cash flow', 'Free cash flow',
       'Income tax paid supplemental data', 'Interest and commission paid',
       'Interest paid supplemental data', 'Investing cash flow',
       'Issuance of capital stock', 'Issuance of debt', 'Operating cash flow',
       'Other cash payments from operating activities',
       'Other cash receipts from operating activities', 'Repayment o

In [ ]:
# Remove columns that have at least 1 NaN value
cash_flow_data_clean = cash_flow_data.dropna(axis=1, how='any')

# For each numeric column, convert to int if all values are whole numbers
for col in cash_flow_data_clean.select_dtypes(include=['float64']).columns:
    if cash_flow_data_clean[col].apply(lambda x: x.is_integer() if pd.notna(x) else True).all():
        cash_flow_data_clean[col] = cash_flow_data_clean[col].astype('Int64')

print(f"Original columns: {len(cash_flow_data.columns)}")
print(f"Cleaned columns: {len(cash_flow_data_clean.columns)}")
print(f"Removed columns: {len(cash_flow_data.columns) - len(cash_flow_data_clean.columns)}")

cash_flow_data_clean.head()

,End cash position,Financing cash flow,Free cash flow,Investing cash flow,industry,sector
ticker,,,,,,
NUF,297358.0,47811.0,-63230.0,-246012.0,Agricultural Inputs,Basic Materials
LLC,621000.0,-532000.0,-783000.0,928000.0,Real Estate - Diversified,Real Estate
NAB,18586000.0,6615000.0,-11720000.0,-264000.0,Banks - Diversified,Financial Services
DXS,65300.0,-690400.0,798100.0,-109600.0,REIT - Office,Real Estate
SPK,34000.0,-593000.0,240000.0,-112000.0,Telecom Services,Communication Services
...,...,...,...,...,...,...
EVN,759542.0,-435814.0,790202.0,-1191581.0,Gold,Basic Materials
DMP,153499.0,-54651.0,91395.0,-54391.0,Restaurants,Consumer Cyclical
IGO,279700.0,-223400.0,37600.0,-4000.0,Other Industrial Metals & Mining,Basic Materials


In [ ]:
# Save the cleaned cash flow data
cash_flow_data_clean.to_csv("combined_data/cash_flow_clean.csv")
print("Saved cleaned cash flow data to combined_data/cash_flow_clean.csv")